In [1]:
#
# super로 부모 클래스를 초기화 해라
#

# 기존에는 자식클래스에서 부모 클래스의 __init__ 메소드를 직접 호출하는 방법으로 부모 클래스를 초기화 했다.
class MyBaseClass:
    def __init__(self, value):
        self.value = value
    
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

In [2]:
# 이 방법은 간단한 모듈에서 작동하지만
# 조금만 복잡해져도 제대로 동작하지 못한다.

# 클래스가 다중 상속(보통 다중상속은 사용해선 안되지만)의 영향 때문에,
# 슈퍼클래스의 __init__ 메소드를 직접 호출하는 방법이 문제를 일으킬 수 있다.

# 한 가지 문제는 __init__의 호출 순서가 명시되어 있지 않다는 점이다.
# 예를 들어 인스턴스의 value 필드로 연산을 수행하는 부모클래스를 두 개 정의해보자

class TimeTwo:
    def __init__(self):
        self.value *= 2
        
class PlusFive:
    def __init__(self):
        self.value += 5

In [3]:
# 그러고 나서 위의 2개의 클래스들을 상속받는 서브클래스를 만든다.

class OneWay(MyBaseClass, TimeTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimeTwo.__init__(self)
        PlusFive.__init__(self)

In [4]:
# 인스턴스를 만들어서 value를 출력해보자
foo = OneWay(5)
print(foo.value)

15


In [5]:
# 예상했던 값이 나왔다.
# MyBaseClass 에서 value = 5 가 정의되고
# TimeTwo 에서 *2를 통해 10이 된다.
# PlusFive를 통해 +5가 되고 결국 15가 출력된다.

In [10]:
# 다음은 다른 순서로 정의한 클래스다
class AnotherWay(MyBaseClass, PlusFive, TimeTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimeTwo.__init__(self)
        PlusFive.__init__(self)

boo = AnotherWay(5)
print(boo.value)

15


In [11]:
# 하지만 위 클래스는 순서에 맞게 출력되지 않는다.
# 본래 예상대로라면, 5를 먼저 더하고 나서 *2를 해서
# 20이 나와야한다는 생각이 든다.

In [13]:
# 이러한 문제는 다이아몬드 상속이다.
# 다이아몬드 상속은 어떤 특정 클래스가
# 동일한 부모클래스를 상속받는 두 개의 클래스를 동시에 상속받을 때 발생한다.

# 동일한 부모클래스의 __init__을 여러번 호출하기 때문에 예상치 못한 동작을 일으킨다.
# 예를 들어 MyBaseClass를 상속받는 두 개의 서브클래스를 정의해보자

class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        

class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

In [14]:
# 다음으로 두 클래스에서 모두 상속받는 자식 클래스를 정의하여
# MyBaseClass를 다이아몬드 꼭대기로 만든다.
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value) # 여기서 25로 만들지만
        PlusTwo.__init__(self, value) # 여기서 MyBaseClass.__init__을 호출하면서 다시 5로 리셋된다.

foo = ThisWay(5)
print('(5x5)+2 라서 27이 나와야하는데 현실은 ', foo.value)

(5x5)+2 라서 27이 나와야하는데 현실은  7


In [17]:
# 27이 나와야할 것 같지만, 두 번째 부모 클래스의 생성자 PlusTwo.__init__을 호출 하는 코드가 있어서
# MyBaseClass.__init__가 두 번째 호출될 때, 25에서 5로 리셋되어버린다.

# python > 3 에서 super를 인수 없이 호출하면, __class__와 self를 인수로 넘겨서 호출한 것으로 처리해서 문제를 해결한다.
class SuperFive(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value)
        self.value *= 5
        
class SuperTwo(MyBaseClass):
    def __init__(self, value):
        super().__init__(value)
        self.value += 2
        
class Result(SuperTwo, SuperFive):
    def __init__(self, value):
        super().__init__(value)

In [18]:
moo = Result(5)
print(moo.value)

27


In [21]:
print(Result.mro())

[<class '__main__.Result'>, <class '__main__.SuperTwo'>, <class '__main__.SuperFive'>, <class '__main__.MyBaseClass'>, <class 'object'>]


In [ ]:
# 인자로 넣어주는 순서에 따라 연산 순서가 변하는데
# mro()에서 보면 알수 있듯, MyBaseClass부터 차례대로 계산된다.
# 5 -> 5*5 -> (5*5)+2
# 27이 결과로 산출된다.